In [25]:
import json
from functools import partial
from traitlets import Any, Dict

from sepal_ui import sepalwidgets as sw
from ipywidgets import Output
import ipyvuetify as v
import pandas as pd

In [26]:
class EditDialog(v.Dialog):
    
    model = Dict().tag(sync=True)
    
    def __init__(self, table, schema,  *args, default = None, **kwargs):
        """
        
        Args: 
            schema (dict {'title':'type'}): Schema for table
            title (str): Title for the dialog
            default (dict): Dictionary with default valules
        """
        
        self.table = table
        self.default = default
        self.title = "New element" if not self.default else "Modify element"
        self.schema = schema
        self.v_model=True
        self.max_width=500
        self.overlay_opcity=0.7
        
        # Action buttons
        self.save = v.Btn(children=['Save'])
        self.cancel = v.Btn(children=['Cancel'])
        self.modify = v.Btn(children=['Modify'])
        
        save = [self.save, self.cancel]
        modify = [self.modify, self.cancel]
        
        actions = v.CardActions(children=save if not default else modify)
        
        super().__init__(*args, **kwargs)
        
        self.children=[
            v.Card(
                class_='pa-4',
                children=[
                    v.CardTitle(children=[self.title])] + \
                    self._get_widgets() + \
                    [actions]
            )
        ] 
        
        # Create events
        
        self.save.on_event('click', self._save)
        self.modify.on_event('click', self._modify)
        self.cancel.on_event('click', self._cancel)
        
    def _modify(self, widget, event, data):
        """Modify elements to the table"""
        
        current_items = self.table.items.copy()
        item_to_modify = self.table.v_model[0]
        
        for i, item in enumerate(current_items):
            if item_to_modify['id'] == self.model['id']:
                current_items[i] = self.model
        
        self.table.items = current_items
        self.v_model=False
    
    def _save(self, widget, event, data):
        """Add elements to the table"""
        
        current_items = self.table.items.copy()
        item_to_add = self.model
        new_items = [item_to_add] + current_items
        
        self.table.items = new_items
        self.v_model=False
        
        
    def _cancel(self, widget, event, data):
        """Close dialog"""
        
        self.v_model=False
    
    def _populate_dict(self, change, title):
        """Populate model with new values"""
        self.model[title] = change['new']
    
    def _get_widgets(self):
        
        widgets = []
        for title, type_ in self.schema.items():
            widget = v.TextField(label=title.capitalize(), type=type_, v_model='')
            widget.observe(partial(self._populate_dict, title=title), 'v_model')
            if title == 'id':
                widget.disabled=True
            if self.default:
                widget.v_model = self.default[title]
            widgets.append(widget)
            
        return widgets
        


In [27]:
class LayerTable(v.DataTable, sw.SepalWidget):
    
    # unicode value to notify a change
    change_model = Any().tag(sync=True)
    
    def __init__(self, *args, **kwargs):
        """
        
        Args: 
            headers_type (dict {'title':'type'}): Headers for table
        """
        
        self.structure = {'id':'number', 'code':'number', 'description':'string'}
        self.edit_dialog_output = Output()
        
        self.headers = [
            {'text': 'Code', 'value': 'code'},
            {'text': 'Description', 'value': 'description'},
        ]
        
        self.items = [
            {
                'id':i,
                'code': c,
                'description': d,
            } for i, c, d in [[1,'asdf','asdf'], [2,'3242','sadfsa'], [3,'sdf','sadfsa'], [4,'4r2','asdfa']]
        ]
        
        self.edit_icon = v.Icon(children=['mdi-pencil'])
        self.delete_icon = v.Icon(children=['mdi-delete'])
        self.add_icon = v.Icon(children=['mdi-plus'])
        
        slot = v.Toolbar(
            class_='d-flex mb-6',
            flat=True, 
            children=[
                self.edit_dialog_output,
                v.ToolbarTitle(children=['Customise']),
                v.Divider(class_='mx-4', inset=True, vertical=True),
                v.Flex(class_='ml-auto', children=[self.edit_icon, self.delete_icon, self.add_icon])
            ]
        )
        
        self.v_slots = [

                { # the pencil for modification
                    'name': 'top',
                    'variable': 'top',
                    'children': [
                        slot,
                    ]
                }
            ]
        self.disable_pagination = True
        self.change_model = 0
        self.v_model = []
        self.item_key = 'id'
        self.show_select = True
        self.single_select = True
        
        
        super().__init__(*args, **kwargs)
        
        # link the search textField 
#         self.edit_icon.on_event('click', self._on_click)
        
        self.edit_icon.on_event('click', self._edit_event)
        self.delete_icon.on_event('click', self._remove_event)
        self.add_icon.on_event('click', self._add_event)
        
#         self.dialog_edit.observe(self._on_dialog_change, 'custom_v_model')
        
    def _edit_event(self, widget, event, data):

        dial = EditDialog(
            schema = self.structure, 
            default= self.v_model[0],
            table = self
        )
        with self.edit_dialog_output:
            display(dial)
                    
    def _add_event(self, widget, event, data):
        
        dial = EditDialog(
            schema = self.structure, 
            table = self
        )
        with self.edit_dialog_output:
            display(dial)
        
    def _remove_event(self, widget, event, data):
        """Remove current selected (self.v_model) element from table"""
        
        current_items = self.items.copy()
        current_items.remove(self.v_model[0])
        
        self.items = current_items


In [28]:
lt = LayerTable()

In [29]:
lt

LayerTable(change_model=0, disable_pagination=True, headers=[{'text': 'Code', 'value': 'code'}, {'text': 'Desc…